In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# read csv
file_path = 'apartment_transaction.txt'
df = pd.read_csv(file_path, encoding='utf-8', sep='|')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# --------------------------------------------------------------------------------
# parameter 이름
# --------------------------------------------------------------------------------

# read csv
file_path = 'apartment_transaction.txt'
df = pd.read_csv(file_path, encoding='utf-8', sep='|')

# PreProcessing
'''
start_date = pd.to_datetime('2006-01-01')
df['dealDate'] = pd.to_datetime(df['dealDate'])
df['dealDate'] = (df['dealDate'] - start_date).dt.days
'''

df['dealDate'] = pd.to_datetime(df['dealDate'])
df['year'] = df['dealDate'].dt.year
df['month'] = df['dealDate'].dt.month
df['day'] = df['dealDate'].dt.day

df['month_sin'] = np.sin(2*np.pi*df['month'] / 12)
df['month_cos'] = np.cos(2*np.pi*df['month'] / 12)
df['day_sin'] = np.sin(2*np.pi*df['day'] / 12)
df['day_cos'] = np.cos(2*np.pi*df['day'] / 12)

df = df.drop(['month', 'day'], axis=1)

df

,dealDate,interestRate,gu,dong,exclusiveArea,floor,buildYear,dealAmount,apartmentName,jibunAddress,year,month_sin,month_cos,day_sin,day_cos
0,2024-10-19,3.25,강남구,대치동,84.580,1,2000,250000,대치삼성,강남구 대치동 1014-3,2024,-0.866025,0.500000,-5.000000e-01,-8.660254e-01
1,2024-10-29,3.25,강남구,청담동,90.197,9,2011,290000,청담자이,강남구 청담동 134-38,2024,-0.866025,0.500000,5.000000e-01,-8.660254e-01
2,2024-10-19,3.25,강남구,개포동,59.888,2,2019,240000,래미안블레스티지,강남구 개포동 1280,2024,-0.866025,0.500000,-5.000000e-01,-8.660254e-01
3,2024-10-26,3.25,강남구,대치동,84.950,20,2008,320000,대치아이파크,강남구 대치동 888,2024,-0.866025,0.500000,8.660254e-01,5.000000e-01
4,2024-10-23,3.25,강남구,대치동,85.000,8,1999,254500,대치현대,강남구 대치동 974,2024,-0.866025,0.500000,-5.000000e-01,8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288183,2006-01-21,3.75,중랑구,신내동,39.840,4,1996,9700,신내6대주,중랑구 신내동 650,2006,0.500000,0.866025,-1.000000e+00,-4.286264e-16
1288184,2006-01-18,3.75,중랑구,신내동,39.840,6,1996,9560,신내6대주,중랑구 신내동 650,2006,0.500000,0.866025,3.673940e-16,-1.000000e+00
1288185,2006-01-03,3.75,중랑구,망우동,51.690,5,2002,9000,현대,중랑구 망우동 508-17,2006,0.500000,0.866025,1.000000e+00,6.123234e-17
1288186,2006-01-04,3.75,중랑구,신내동,59.400,7,1993,12750,동성1,중랑구 신내동 397,2006,0.500000,0.866025,8.660254e-01,-5.000000e-01


In [3]:
# Nominals
from sklearn.preprocessing import LabelEncoder

df = df.drop(['jibunAddress'], axis=1)

le_gu = LabelEncoder()
le_dong = LabelEncoder()


df['gu'] = le_gu.fit_transform(df['gu'])
df['dong'] = le_dong.fit_transform(df['dong'])

In [4]:
import pickle
with open('le_gu.pkl', 'wb') as file:
    pickle.dump(le_gu, file)

In [5]:
import pickle
with open('le_dong.pkl', 'wb') as file:
    pickle.dump(le_dong, file)

In [6]:
# Create Model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

X = df.drop(['dealDate', 'dealAmount', 'apartmentName'], axis=1)
y = df['dealAmount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
X

,interestRate,gu,dong,exclusiveArea,floor,buildYear,year,month_sin,month_cos,day_sin,day_cos
0,3.25,0,52,84.580,1,2000,2024,-0.866025,0.500000,-5.000000e-01,-8.660254e-01
1,3.25,0,259,90.197,9,2011,2024,-0.866025,0.500000,5.000000e-01,-8.660254e-01
2,3.25,0,8,59.888,2,2019,2024,-0.866025,0.500000,-5.000000e-01,-8.660254e-01
3,3.25,0,52,84.950,20,2008,2024,-0.866025,0.500000,8.660254e-01,5.000000e-01
4,3.25,0,52,85.000,8,1999,2024,-0.866025,0.500000,-5.000000e-01,8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...
1288183,3.75,23,158,39.840,4,1996,2006,0.500000,0.866025,-1.000000e+00,-4.286264e-16
1288184,3.75,23,158,39.840,6,1996,2006,0.500000,0.866025,3.673940e-16,-1.000000e+00
1288185,3.75,23,83,51.690,5,2002,2006,0.500000,0.866025,1.000000e+00,6.123234e-17
1288186,3.75,23,158,59.400,7,1993,2006,0.500000,0.866025,8.660254e-01,-5.000000e-01


In [8]:
model = RandomForestRegressor(n_estimators=30, random_state=0, max_depth=20)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}, R2: {r2}')

MSE: 80139895.4381418, R2: 0.9681066555762379


In [10]:
importances = model.feature_importances_
feature_names = X.columns

for name, importance in zip(feature_names, importances):
    print(f"{name}: {importance:.4f}")

interestRate: 0.0086
gu: 0.1981
dong: 0.0607
exclusiveArea: 0.3711
floor: 0.0120
buildYear: 0.1186
year: 0.2215
month_sin: 0.0032
month_cos: 0.0026
day_sin: 0.0018
day_cos: 0.0017


In [11]:
import os
import matplotlib.pyplot as plt

X_test_with_dates = df.loc[X_test.index]  # 원래 인덱스를 사용하여 X_test와 같은 데이터 선택

# gu는 가나다순 배열.
for target_gu in range(0, 24):
    output_dir = 'results/result_images_' + le_gu.inverse_transform([target_gu])[0]
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    results = pd.DataFrame({'Date': X_test_with_dates['dealDate'], 'gu': X_test_with_dates['gu'],
                            'dong': X_test_with_dates['dong'], 'Actual': y_test, 'Predicted': y_pred})
    
    results = results[results['gu']==target_gu]
    
    # int->datetime
    results.set_index('Date', inplace=True)
    chunks = results.groupby(pd.Grouper(freq='3M'))
    
    # 각 그룹에 대해 그래프를 그리고 저장하기
    for i, (name, chunk) in enumerate(chunks):
        plt.figure(figsize=(10, 6))
        
        # 실제 값 (y_test)
        plt.plot(chunk.index, chunk['Actual'], label='Actual Prices', color='blue')
        
        # 예측 값 (y_pred)
        plt.plot(chunk.index, chunk['Predicted'], label='Predicted Prices', color='red', linestyle='--')
    
        # 그래프 꾸미기
        plt.xlabel('Date')
        plt.ylabel('Price (백만원)')
        plt.title(f'Actual vs Predicted Prices (3-month chunk {i+1})')
        plt.legend()
    
        plt.xticks(rotation=45)
        plt.tight_layout()
    
        plt.savefig(os.path.join(output_dir, f'actual_vs_predicted_chunk_{i+1}.png'))
        plt.close()

C:\Users\besta\AppData\Local\Temp\ipykernel_6180\2205843374.py:37: UserWarning: Glyph 48177 (\N{HANGUL SYLLABLE BAEG}) missing from current font.
  plt.tight_layout()
C:\Users\besta\AppData\Local\Temp\ipykernel_6180\2205843374.py:37: UserWarning: Glyph 47564 (\N{HANGUL SYLLABLE MAN}) missing from current font.
  plt.tight_layout()
C:\Users\besta\AppData\Local\Temp\ipykernel_6180\2205843374.py:37: UserWarning: Glyph 50896 (\N{HANGUL SYLLABLE WEON}) missing from current font.
  plt.tight_layout()
C:\Users\besta\AppData\Local\Temp\ipykernel_6180\2205843374.py:39: UserWarning: Glyph 48177 (\N{HANGUL SYLLABLE BAEG}) missing from current font.
  plt.savefig(os.path.join(output_dir, f'actual_vs_predicted_chunk_{i+1}.png'))
C:\Users\besta\AppData\Local\Temp\ipykernel_6180\2205843374.py:39: UserWarning: Glyph 47564 (\N{HANGUL SYLLABLE MAN}) missing from current font.
  plt.savefig(os.path.join(output_dir, f'actual_vs_predicted_chunk_{i+1}.png'))
C:\Users\besta\AppData\Local\Temp\ipykernel_6180\

In [ ]:
import os
import matplotlib.pyplot as plt

X_test_with_dates = df.loc[X_test.index]  # 원래 인덱스를 사용하여 X_test와 같은 데이터 선택

# gu는 가나다순 배열.
for target_gu in range(0, 24):
    output_dir = 'results/result_images_' + le_gu.inverse_transform([target_gu])[0]
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    results = pd.DataFrame({'Date': X_test_with_dates['dealDate'], 'gu': X_test_with_dates['gu'],
                            'dong': X_test_with_dates['dong'], 'Actual': y_test, 'Predicted': y_pred})
    
    results = results[results['gu']==target_gu]
    
    # int->datetime
    results.set_index('Date', inplace=True)
    chunks = results.groupby(pd.Grouper(freq='3M'))
    
    # 각 그룹에 대해 그래프를 그리고 저장하기
    for i, (name, chunk) in enumerate(chunks):
        plt.figure(figsize=(10, 6))
        
        # 실제 값 (y_test)
        plt.plot(chunk.index, chunk['Actual'], label='Actual Prices', color='blue')
        
        # 예측 값 (y_pred)
        plt.plot(chunk.index, chunk['Predicted'], label='Predicted Prices', color='red', linestyle='--')
    
        # 그래프 꾸미기
        plt.xlabel('Date')
        plt.ylabel('Price (백만원)')
        plt.title(f'Actual vs Predicted Prices (3-month chunk {i+1})')
        plt.legend()
    
        plt.xticks(rotation=45)
        plt.tight_layout()
    
        plt.savefig(os.path.join(output_dir, f'actual_vs_predicted_chunk_{i+1}.png'))
        plt.close()

In [12]:
results.to_csv('Predicted_Price.csv', index=True, encoding='utf-8')

In [9]:
import joblib
joblib.dump(model, 'random_forest.joblib')

['random_forest.joblib']